In [2]:
import sys
import os
import numpy as np
import pandas as pd
import pickle
import psutil

sys.path.insert(1,'/data1/gridsan/groups/manthiram_lab/Utils')
sys.path.insert(1,'/data1/gridsan/groups/manthiram_lab/QMkit')

from utils import make_xyz_from_array
from dft_utils import mol_with_atom_index #Need to check this function and replace with the one from dft_utils
from dft_script_gen import gaussian,gen_gauss_sub_script
from gen_3D import make_smi_file, get_xyz_from_smiles_string, get_xyz_from_mol, get_xyz_from_xyz_arr,GetCoordsFromMolBlock

import utils
from utils import make_xyz_from_array, GetCoordsFromMolBlock
import xyz2mol as x2m
import re
import importlib
from tqdm import tqdm
import shutil
import glob
import pathlib

In [4]:
bases=[['carbonate','O=C([O-])O','O=C(O)O']]

In [7]:
actually_make_dirs=True
if actually_make_dirs:

    cwd=os.getcwd()

    #for i, s in enumerate(tqdm(y)):

    functional='M062X'
    basis= 'Def2SVPD'
    solvorgas='solv' #This should only be used for gas phase, since we need the gas phase pre opt for solv
    solvmethod='SMD'
    solvent='acetonitrile'
    #cwd='/home2/gridsan/jmaalouf/vHTP/Code/ylide/Library'
    shared_dir='/data1/gridsan/groups/manthiram_lab/vHTP/Code/ylide/Library/'
    
    for i, s in enumerate(tqdm(bases)):

        assert solvorgas == 'gas' or solvorgas=='solv','SOLVORGAS NOT SET TO A VALID VALUE'
        
        p_dict={
            0: 'name',
            1: 'deprot',
            2: 'prot'
        }
        
        for j, b in enumerate(s[1:]): 
            if solvorgas=='gas':

                #basepath=f'{cwd}/{calc_folder}/{str(i).zfill(z)}/ylide{suff}/{functional}/{basis}/gas'
                basepath=f'{shared_dir}/bases/{s[0]}/{p_dict[j+1]}/{functional}/{basis}/gas/'

            elif solvorgas=='solv':

                try:
                    solvent
                except NameError:
                    raise Exception('PLEASE SPECIFY A SOLVENT')


                #basepath=f'{cwd}/{calc_folder}/{str(i).zfill(z)}/ylide{suff}/{functional}/{basis}/{solvent}/{solvmethod}'
                basepath=f'{shared_dir}/bases/{s[0]}/{p_dict[j+1]}/{functional}/{basis}/{solvent}/{solvmethod}/'
                
            os.makedirs(basepath,exist_ok=True)
            
            smi= b
            path=f'{basepath}/{p_dict[j+1]}'

            try:
                make_smi_file(smi,path)
                get_xyz_from_smiles_string(smi,path,numConfs=100)
                gaussian(path,smi,functional=functional,basis=basis,solvorgas=solvorgas,solvmethod=solvmethod,solvent=solvent)
                gen_gauss_sub_script(path)
            except Exception as e:
                print(e)
                exc_type, exc_obj, exc_tb = sys.exc_info()
                fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                print(exc_type, fname, exc_tb.tb_lineno)
                print(f'ERROR WITH  MOLECULE NUMBER {i} smiles {path}')

100%|██████████| 1/1 [00:00<00:00,  1.58it/s]


In [27]:
functional='M062X'
basis= 'Def2TZVP'
solvorgas='solv'
solvmethod='SMD'
solvent='acetonitrile'

submit=False
#os.chdir(orig_dir)

p_dict={
    0: 'name',
    1: 'deprot',
    2: 'prot'
}

if submit:
    
    for i,s in enumerate(bases):

        for j,b in enumerate(s[1:]):


            assert solvorgas == 'gas' or solvorgas=='solv','SOLVORGAS NOT SET TO A VALID VALUE'

            if solvorgas=='gas':
                path=pathlib.Path(os.path.join(shared_dir, 'bases', s[0],p_dict[j+1], functional, basis,'gas', p_dict[j+1]))

            elif solvorgas=='solv':
                path=pathlib.Path(os.path.join(shared_dir, 'bases' ,s[0],p_dict[j+1], functional, basis,solvent,solvmethod, p_dict[j+1]))

            par_path=path.parent.absolute()
            filename=os.path.basename(os.path.normpath(path))

            #GET LOG FILE NAME
            #split_file=filename.split('.')
            log_file=f'{path}.log'
            
            os.chdir(par_path)
            subprocess.run(f'sbatch {filename}.sh',shell=True)
            os.chdir(cwd)
#             if exists(log_file)==False:

#                 os.chdir(par_path)
#                 subprocess.run(f'sbatch {filename}.sh',shell=True)
#                 os.chdir(orig_dir)